## 必要なモジュールのインストール

In [1]:
import sys
import os
sys.path.append(os.pardir)
import pandas as np
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
from utils import label_to_one_hot
from utils import ComputeTfidf
from sklearn.decomposition import TruncatedSVD

## 学習/テストデータの読み込み

In [2]:
with open('../data/train_data.pickle', 'rb') as rb:
    train_data = pickle.load(rb)

with open('../data/test_data.pickle', 'rb') as rb:
    test_data = pickle.load(rb)

### データ確認

In [3]:
df_train = pd.DataFrame(train_data, columns=['label', 'text'])
df_test = pd.DataFrame(test_data, columns=['label', 'text'])

print('train_data')
display(df_train.head())
display(df_train['label'].value_counts())
print()
print('test_data')
display(df_test.head())
display(df_test['label'].value_counts())

train_data


,label,text
0,本会議,会議 開く\n本日 参議院 本 院 送付 テロ対策海上阻止活動に対する補給支援活動の実施に関...
1,総務委員会,会議 開く\n行政 機構 運営 件 公務員 制度 給与 恩給 件 地方自治 地方税 財政 件...
2,議院運営委員会,会議 開く\nまず 本日 参議院 テロ対策海上阻止活動に対する補給支援活動の実施に関する特別...
3,厚生労働委員会,会議 開く\n谷垣禎一 君 提出 特定 フィブリノゲン製剤 特定 血液 凝固 第9 因子 製...
4,本会議,会議 開く\n議案 上程 緊急 動議 提出 いたす\n厚生 労働 委員長 提出 特定 フィブ...


議院運営委員会      1053
本会議           985
厚生労働委員会       644
予算委員会         562
総務委員会         531
法務委員会         518
国土交通委員会       505
経済産業委員会       477
内閣委員会         470
農林水産委員会       431
環境委員会         363
財務金融委員会       267
外務委員会         213
文部科学委員会       210
安全保障委員会       124
決算行政監視委員会      90
Name: label, dtype: int64


test_data


,label,text
0,議院運営委員会,会議 開く\nまず 庶務 小 委員長 報告 発言 求める られる おる 許す\n石田真敏 君...
1,財務金融委員会,会議 開く\n内閣 提出 所得税法 一部 改正 する 法律案 議題 いたす\n諮る いたす\...
2,内閣委員会,会議 開く\n内閣 提出 特定 複合 観光施設 区域 整備 法案 議題 いたす\n諮る いた...
3,文部科学委員会,会議 開く\n内閣 提出 文化財保護法 地方教育行政の組織及び運営に関する法律 一部 改正 ...
4,安全保障委員会,会議 開く\n安全保障 件 特に イラク 派遣 日報 調査 進める\n防衛大臣 報告 聴取 ...


議院運営委員会      124
本会議          105
厚生労働委員会       81
内閣委員会         76
農林水産委員会       69
法務委員会         63
国土交通委員会       56
予算委員会         55
経済産業委員会       38
総務委員会         36
環境委員会         32
文部科学委員会       26
外務委員会         22
財務金融委員会       20
安全保障委員会       14
決算行政監視委員会      5
Name: label, dtype: int64

### ラベルとテキストに分割

In [4]:
x_train_text = [x[1] for x in train_data]
y_train = [x[0] for x in train_data]
x_test_text = [x[1] for x in test_data]
y_test = [x[0] for x in test_data]

#テキストにてtf-idfモデルを行う都合上、テキストを統合する。
train_size, test_size = len(x_train_text), len(x_test_text)
print('train_size: %i' % train_size)
print('test_size: %i' % test_size)
x_corpus = x_train_text + x_test_text

train_size: 7443
test_size: 822


## テキストをベクトルに変換する
tf-idfを用いた変換を実施する。プログラムについてはutilsに記述したプログラムを呼び出して用いる。  
ここではsklearnのCountVectorizer、TfidfTransformerを用いてtf-idfを計算している。  
今回は一つの会議の全発言を文書単位としてtf-idf値を算出した。  

ここでsklearnのtf-idf値の算出ロジックについて言及する。  
tf-idf値はtfとidf値の積で計算され、文書数$ \times $コーパスを構成する単語数の行列で行成される。  
いま全体のコーパスを$ D $、コーパスを構成する文書を$ d_{i} $、文書に含まれる単語を$ t_{j} $とする。  
いまコーパスの文書数を$ m $、文書に含まれる単語数を$ n $とすると、コーパス$ D $は
$$
D: \{ d_{1}, d_{2},...,d_{m} \} \\
D: \{ t_{1}, t_{2},...,t_{n} \}
$$
とかける。

tf値は文書ごとの単語の構成割合である。$ d_{i} $が$p$種類の単語$ t_{i1}, t_{i2}, ..., t_{ip} $で構成されているとき(ただしそれぞれの単語は2個以上存在することもありうる)、
$$
 \textrm{tf}_{i, j} = \frac{|t_{ij}|}{\sum_{l=1}^{p} |t_{il}|}
$$
である。ただし$ | \cdot | $は$  \cdot  $の要素の個数となる。  
idf値は単語の文書の出現割合から算出される。単語$ t_{j} $のidf値は
$$
\textrm{idf}_{j} = \log \left(  \frac{m+1}{|\{ d: d \ni t_{j}  \} | + 1} \right)
$$

ただし$|\{ d: d \ni t_{j}  \} |$は単語jが含まれる文書の数である。よって$\textrm{tfidf}_{i,j}$は
$$
\textrm{tfidf}_{i,j} = \textrm{tf}_{i, j} \times \textrm{idf}_{j}
$$
となる。
TfidfTransformerを用いた場合文書数$ \times$単語数$ \left( m \times n \right)$の行列
$$
\left(
\begin{array}{cccc}
\textrm{tfidf}_{1,1} & \textrm{tfidf}_{1,2} & ... & \textrm{tfidf}_{1,n} \\
\textrm{tfidf}_{2,1} & \textrm{tfidf}_{2,2} & ... & \textrm{tfidf}_{2,n} \\
... \\
\textrm{tfidf}_{m,1} & \textrm{tfidf}_{m,2} & ... & \textrm{tfidf}_{m,n} \\
\end{array}
\right)
$$
で算出される。また算出結果は列ごとに規格化されて出力される。
$$
 \sqrt{\sum_{j=1}^{n} \textrm{tfidf}_{i,j}^2 } = 1
$$

In [5]:
ct = ComputeTfidf(x_corpus)

In [6]:
ct.compute_tf()
text_tfidf = ct.compute_tfidf()
print(text_tfidf.shape)

(8265, 150053)


## 特異値分解(SVD)による次元圧縮
上記で求めたtfidf値を次元圧縮する

In [7]:
svd = TruncatedSVD(n_components=2048, n_iter=5)
svd.fit(text_tfidf)
text_tfidf_svd = svd.fit_transform(text_tfidf)

In [8]:
print(text_tfidf_svd.shape)
print('寄与率 %f' % np.sum(svd.explained_variance_ratio_))
svd.explained_variance_ratio_

(8265, 2048)
寄与率 0.889716


array([1.03423309e-01, 7.01510859e-02, 3.23799627e-02, ...,
       5.24560759e-05, 5.24401771e-05, 5.23903279e-05])

### 学習用ファイルとテスト用ファイルに分解する

In [9]:
x_train = text_tfidf_svd[:train_size,:]
x_test = text_tfidf_svd[train_size:,:]
print(len(x_train))
print(len(x_test))

7443
822


## ラベルをIDに置き換える
今回分析対象の委員会を以下の通り通し番号をつける。

|委員会 |番号|
|---|---|
|本会議|0|
|内閣委員会|1|
|総務委員会|2|
|法務委員会|3|
|外務委員会|4|
|財務金融委員会|5|
|文部科学委員会|6|
|厚生労働委員会|7|
|農林水産委員会|8|
|経済産業委員会|9|
|国土交通委員会|10|
|環境委員会|11|
|安全保障委員会|12|
|予算委員会|13|
|決算行政監視委員会|14|
|議院運営委員会|15|



In [10]:
classify_dict = {'本会議':0, '内閣委員会':1, '総務委員会':2, '法務委員会':3, '外務委員会':4, '財務金融委員会':5, '文部科学委員会':6, '厚生労働委員会':7, '農林水産委員会':8, \
                '経済産業委員会':9, '国土交通委員会':10, '環境委員会':11, '安全保障委員会':12, '予算委員会':13, '決算行政監視委員会':14, '議院運営委員会':15}
classify_size = len(classify_dict)
y_train = np.array([classify_dict[x] for x in tqdm_notebook(y_train)])
y_test = np.array([classify_dict[x] for x in tqdm_notebook(y_test)])

y_train

array([ 0,  2, 15, ...,  8,  3, 15])

## クラスの重みを算出する
分析対象の委員会のデータ数のばらつきが大きいため、少ないクラス(委員会)の学習時のコストを大きくする。

In [11]:
labels = np.arange(16)
cost_weight = np.zeros((16))
for label in labels:
    cnt = np.sum(y_train==label)
    cost_weight[label] = cnt

max_cnt = np.max(cost_weight)
cost_weight= np.reciprocal(cost_weight) * max_cnt
cost_weight = {int(label):cost_weight[label] for label in labels}
cost_weight

{0: 1.069035532994924,
 1: 2.2404255319148936,
 2: 1.983050847457627,
 3: 2.032818532818533,
 4: 4.943661971830986,
 5: 3.943820224719101,
 6: 5.014285714285715,
 7: 1.6350931677018632,
 8: 2.4431554524361947,
 9: 2.207547169811321,
 10: 2.0851485148514852,
 11: 2.9008264462809916,
 12: 8.491935483870968,
 13: 1.873665480427046,
 14: 11.700000000000001,
 15: 1.0}

### One-hotベクトルの作成
上記通し番号に沿って、one-hotベクトルに変換する。以下に法務委員会の場合のone-hotベクトルを示す。
$$
\left(\begin{array}{c}
            0 \\
            0 \\
            0 \\
            1 \\
            0 \\
            ... \\
            0 \\
        \end{array}\right) \quad
$$

In [12]:
classify_size = len(classify_dict)
y_train = np.array([label_to_one_hot(classify_size, x) for x in tqdm_notebook(y_train)])
y_test = np.array([label_to_one_hot(classify_size ,x) for x in tqdm_notebook(y_test)])

In [13]:
y_train

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [14]:
with open('train_data.pickle', 'wb') as wb:
    pickle.dump((x_train, y_train), wb)
    
with open('test_data.pickle', 'wb') as wb:
    pickle.dump((x_test, y_test), wb)
    
with open('cost_weight.pickle', 'wb') as wb:
    pickle.dump(cost_weight, wb)